In [ ]:
pip install tensorflow

In [2]:
# --------------------------
# Fix truncated images
# --------------------------
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --------------------------
# Imports
# --------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from sklearn.utils import shuffle
from datetime import datetime
from tensorflow.keras import mixed_precision

# --------------------------
# Mixed precision
# --------------------------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --------------------------
# Paths
# --------------------------
train_dir = r"E:\Engineering\5 th sem\Project Sem 5\HPC\Dataset\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\train"
val_dir   = r"E:\Engineering\5 th sem\Project Sem 5\HPC\Dataset\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\val"
test_dir  = r"E:\Engineering\5 th sem\Project Sem 5\HPC\Dataset\Bone_Fracture_Binary_Classification\Bone_Fracture_Binary_Classification\test"

# --------------------------
# Parameters
# --------------------------
classes = ["fractured", "not fractured"]  # EXACT folder names
img_size = (160, 160)
batch_size = 64

# --------------------------
# Prepare true balanced training generator
# --------------------------
file_paths = {cls: [os.path.join(train_dir, cls, f) for f in os.listdir(os.path.join(train_dir, cls))] for cls in classes}
max_count = max(len(file_paths[cls]) for cls in classes)

# Oversample minority class
for cls in classes:
    n = len(file_paths[cls])
    if n < max_count:
        extra = np.random.choice(file_paths[cls], max_count - n, replace=True)
        file_paths[cls].extend(extra)

# Flatten and shuffle
all_files = []
all_labels = []
for i, cls in enumerate(classes):
    all_files.extend(file_paths[cls])
    all_labels.extend([i] * len(file_paths[cls]))
all_files, all_labels = shuffle(all_files, all_labels, random_state=42)

# Generator
def balanced_generator(file_paths, labels, batch_size):
    while True:
        for i in range(0, len(file_paths), batch_size):
            batch_files = file_paths[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            batch_images = []
            for f in batch_files:
                img = tf.keras.preprocessing.image.load_img(f, target_size=img_size)
                x = tf.keras.preprocessing.image.img_to_array(img)
                x = preprocess_input(x)
                batch_images.append(x)
            yield np.array(batch_images), np.array(batch_labels)

train_gen = balanced_generator(all_files, all_labels, batch_size)
steps_per_epoch = len(all_files) // batch_size

# --------------------------
# Validation & test generators
# --------------------------
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
test_gen = val_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False)

# --------------------------
# DenseNet121 model
# --------------------------
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size+(3,))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid', dtype='float32')
])

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --------------------------
# Callbacks
# --------------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/fracture_model_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# --------------------------
# Train Phase 1: Frozen base
# --------------------------
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=len(val_gen),
    epochs=7,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Fine-tune last 50 layers
# --------------------------
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

# model.compile(
#     optimizer=optimizers.Adam(1e-5),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

# history2 = model.fit(
#     train_gen,
#     validation_data=val_gen,
#     steps_per_epoch=steps_per_epoch,
#     validation_steps=len(val_gen),
#     epochs=10,
#     callbacks=callbacks,
#     verbose=1
# )

# --------------------------
# Save final model with timestamp
# --------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_name = f"fracture_{timestamp}.keras"
model.save(final_model_name)
print(f"Final model saved as {final_model_name}")

# --------------------------
# Evaluate on test set
# --------------------------
loss, acc = model.evaluate(test_gen)
print(f"\nFinal Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")



Found 829 images belonging to 2 classes.
Found 506 images belonging to 2 classes.
Epoch 1/7
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4922 - loss: 1.0425

c:\Users\Siri Priya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\Siri Priya\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch 1: val_accuracy improved from None to 0.51267, saving model to /kaggle/working/fracture_model_best.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 813s 6s/step - accuracy: 0.5056 - loss: 0.9988 - val_accuracy: 0.5127 - val_loss: 0.7659 - learning_rate: 1.0000e-04
Epoch 2/7
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5398 - loss: 0.8950
Epoch 2: val_accuracy improved from 0.51267 to 0.62485, saving model to /kaggle/working/fracture_model_best.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 779s 5s/step - accuracy: 0.5527 - loss: 0.8614 - val_accuracy: 0.6248 - val_loss: 0.6570 - learning_rate: 1.0000e-04
Epoch 3/7
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6026 - loss: 0.7757
Epoch 3: val_accuracy improved from 0.62485 to 0.70205, saving model to /kaggle/working/fracture_model_best.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 795s 5s/step - accuracy: 0.5972 - loss: 0.7767 - val_accuracy: 0.7021 - val_loss: 0.5912 - learning_rate: 1.0000e-04
Epoch 4/7
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - ac